In [5]:
import CalculatedFieldSubroutines as cfs

import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

from pandasgui import show

import matplotlib.pyplot as plt

import seaborn as sns

import warnings

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from IPython.display import clear_output
#import in neede libraries

In [6]:
warnings.filterwarnings( 'ignore' )

In [7]:
gmID_list = cfs.list_whitelisted_gmIDs()

topic_list = cfs.list_topics()

print( topic_list )

FileNotFoundError: [Errno 2] No such file or directory: '/home/jle_linux/Desktop/TDMprivate/data'

In [ ]:
red_combined_dfs = []

for gmID in gmID_list:

    if ( cfs.give_route( gmID ) != 'Red' ):

        continue

    chassis_df = cfs.retrieve_gmID_topic( gmID, '/apollo/canbus/chassis' )

    best_pose_df = cfs.retrieve_gmID_topic( gmID, '/apollo/sensor/gnss/best/pose' )

    chassis_df.sort_values( 'time' )

    best_pose_df.sort_values( 'time' )

    #

    cfs.BinaryDrivingMode( chassis_df )

    cfs.TernaryDrivingModeTransition( chassis_df )

    cfs.DistanceToNearestDisengagement( chassis_df )

    #

    cfs.LatLonTotalStdDev( best_pose_df )

    cfs.ProgressAlongRoute_v2( best_pose_df )

    #

    cfs.ChassisBestPoseMatchedTime( chassis_df, best_pose_df )

    #

    red_combined_df = pd.merge( chassis_df, best_pose_df, on = 'ChassisBestPoseMatchedTime', how = 'inner' )

    #

    red_combined_df = red_combined_df[ [ 'brakePercentage', 'speedMps', 'time_x', 'throttlePercentage', 'steeringPercentage', \
                                 'TernaryDrivingModeTransition', 'DistanceToNearestDisengagement', 'longitude', 'latitude', \
                                 'LatLonTotalStdDev', 'ProgressAlongRoute', 'PartitionNumber' ] ]

    red_combined_dfs.append( red_combined_df )

In [ ]:
red_combined_df_all = pd.concat( red_combined_dfs )

In [ ]:
red_combined_df_all_disengagements = red_combined_df_all[ red_combined_df_all[ 'TernaryDrivingModeTransition' ] == -1 ]


In [ ]:
data = red_combined_df[ [ 'brakePercentage', 'speedMps', 'throttlePercentage', 'steeringPercentage' ] ].copy()


In [ ]:
data

In [ ]:
data = ((data - data.min())/(data.max()-data.min())) * 9 + 1

In [ ]:
data.describe()

In [ ]:
def random_centroids(data,k):
    centroids = []
    for i in range(k):
        centroid = data.apply(lambda x: float(x.sample()))
        centroids.append(centroid)
    return pd.concat(centroids, axis=1)
#a function that will take in the number of clusters we want with our data
#initialize a list called centroids for i in range k
#then add it to the list of centroids
#the number of centroids is equivalent to the number of clusters we want
#combines all of the centroids(individual pandas series) into a data frame

In [ ]:
centroids = random_centroids(data,5)
#we can now run the random_centroids

In [ ]:
centroids

In [ ]:
def get_labels(data, centroids):
    distances = centroids.apply(lambda x: np.sqrt(((data - x) ** 2).sum(axis=1)))
    return distances.idxmin(axis=1)
#label each data point 
#finds the euclidean distance between a data point and each cluster for all centroids 
#finding which cluster center is closest to that point 

In [ ]:
labels = get_labels(data, centroids)

In [ ]:
labels.value_counts()
#how many times each value occurs 

In [ ]:
def new_centroids(data, labels, k):
    centroids = data.groupby(labels).apply(lambda x: np.exp(np.log(x).mean())).T
    return centroids
#splits the data by each cluster and then calculate a new center for each group

In [ ]:
def plot_clusters(data, labels, centroids, iteration):
    pca = PCA(n_components=2)
    data_2d = pca.fit_transform(data)
    centroids_2d = pca.transform(centroids.T)
    clear_output(wait=True)
    plt.title(f'Iteration {iteration}')
    plt.scatter(x=data_2d[:,0], y=data_2d[:,1], c=labels)
    plt.scatter(x=centroids_2d[:,0], y=centroids_2d[:,1])
    plt.show()
#Using PCA to change the multidimensional values into 2-D data
#Pass in the several column data and get it converted into 2 column data
#plotting the clusters and coloring from the plotting of centroids 
#will wait and change the graph over the already completed graph 

In [ ]:
max_iterations = 100
centroid_count = 5

centroids = random_centroids(data, centroid_count)
old_centroids = pd.DataFrame()
iteration = 1

while iteration < max_iterations and not centroids.equals(old_centroids):
    old_centroids = centroids
    
    labels = get_labels(data, centroids)
    centroids = new_centroids(data, labels, centroid_count)
    plot_clusters(data, labels, centroids, iteration)
    iteration += 1
#when new centroids are equilavent to past centroids (indicates that it has stopped shifting)
#update the data to see which label corresponds with each cluster

In [ ]:
c0.describe()

In [ ]:
c1.describe()

In [ ]:
c2.describe()

In [ ]:
import matplotlib.pyplot as plt

# Define cluster data and colors
clusters = [c0, c1, c2, c3, c4]
colors = ['red', 'blue', 'green', 'purple', 'orange']
labels = ['Cluster 0', 'Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4']

# Initialize plot
plt.figure(figsize=(10, 7))

# Plot each cluster using latitude and longitude
for i, cluster in enumerate(clusters):
    plt.scatter(cluster['longitude'], cluster['latitude'], 
                color=colors[i], label=labels[i], alpha=0.1)

# Add titles and labels
plt.title("Scatterplot: Latitude vs Longitude Across Clusters")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.grid(alpha=0.3)

# Show plot
plt.show()
